<a href="https://colab.research.google.com/github/psychoanalyzing-twitter/sentiment-analysis/blob/main/cw1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
print("Hello World")

Hello World
